<a href="https://colab.research.google.com/github/NwaObed/deeplearning/blob/main/cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
from torchvision.transforms import transforms
import torchvision.models as model
import torch.nn as nn

from google.colab import drive

#Device config
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
transform = transforms.Compose([transforms.ToTensor()])
                                # transforms.Normalize((0.49,0.48,0.45),
                                #                      (0.25,0.24,0.26))])

In [16]:
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        download=True,
                                        train=True,
                                        transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data',
                                        download=True,
                                        train=False,
                                        transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
trainloader = torch.utils.data.DataLoader(trainset,
                                                batch_size=1024,
                                                #num_workers=4,
                                                shuffle=True)

testloader = torch.utils.data.DataLoader(testset,
                                               batch_size=100,
                                               num_workers=4,
                                               shuffle=False)

In [6]:
netModel = model.vgg16()

In [7]:
print(netModel)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
class Identity(nn.Module):
  def __init__(self):
    super(Identity, self).__init__()

  def forward(self, x):
    return x

In [9]:
# Freezing some layers

netModel.avgpool = Identity()
netModel.classifier = nn.Linear(512, 10)

In [47]:
print(netModel)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
# Loss and optimization

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(netModel.parameters(), lr=1e-3)

In [20]:
# Training

netModel.to(device)
netModel.train()

for epoch in range(5):
  losses = []
  for idx, data in enumerate(trainloader, 0):
    image, label = data
    image = image.to(device)
    label = label.to(device)

    output = netModel(image)
    loss = criterion(output, label)
    losses.append(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print('Average loss in epoch {} is {}'.format(epoch, sum(losses)/len(losses)))

Average loss in epoch 0 is 1.4821255207061768
Average loss in epoch 1 is 1.4016079902648926
Average loss in epoch 2 is 1.1997849941253662
Average loss in epoch 3 is 1.0367920398712158
Average loss in epoch 4 is 0.9056236147880554
